In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd


In [ ]:
train_set = "/content/gdrive/MyDrive/CMPE 295A/SQuAD dataset/train-v2.0.json"
train_data = pd.read_json(train_set)
train_data.head()

,version,data
0,v2.0,"{'title': 'Beyoncé', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Frédéric_Chopin', 'paragraphs': [{'..."
2,v2.0,{'title': 'Sino-Tibetan_relations_during_the_M...
3,v2.0,"{'title': 'IPod', 'paragraphs': [{'qas': [{'qu..."
4,v2.0,{'title': 'The_Legend_of_Zelda:_Twilight_Princ...


In [ ]:
test_set = "/content/gdrive/MyDrive/CMPE 295A/SQuAD dataset/dev-v2.0.json"
test_data = pd.read_json(test_set)
test_data.head()

,version,data
0,v2.0,"{'title': 'Normans', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Computational_complexity_theory', '..."
2,v2.0,"{'title': 'Southern_California', 'paragraphs':..."
3,v2.0,"{'title': 'Sky_(United_Kingdom)', 'paragraphs'..."
4,v2.0,"{'title': 'Victoria_(Australia)', 'paragraphs'..."


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   version  442 non-null    object
 1   data     442 non-null    object
dtypes: object(2)
memory usage: 7.0+ KB


In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   version  35 non-null     object
 1   data     35 non-null     object
dtypes: object(2)
memory usage: 688.0+ bytes


In [ ]:
train_data.describe()

,version,data
count,442,442
unique,1,442
top,v2.0,"{'title': '51st_state', 'paragraphs': [{'qas':..."
freq,442,1


In [ ]:
import collections
import json

import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud
import numpy as np


import re
import nltk


from textblob import TextBlob

import spacy

from sklearn import feature_extraction, manifold

import gensim.downloader as gensim_api

import gensim

In [ ]:
sample = train_data['data'][0]['paragraphs'][0]

sample

{'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'qas': [{'answers': [{'answer_start': 269, 'text': 'in the late 1990s'}],
   'id': '56be85543aeaaa14008c9063',
   'is_impossible': False,
   'question': 'When did Beyonce start becoming popular?'},
  {'answers': [{'answer_start': 207, 'text': 'singing and dancing'}],
   'id': '56be8554

In [ ]:
for i in sample:
  print(i)

qas
context


In [ ]:
sample_qas = sample['qas']
sample_context = sample['context']
sample_context

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [ ]:
pip install langdetect

In [ ]:
import langdetect 
from langdetect import detect

In [ ]:
print(detect(sample_context))



en


In [ ]:
#txt = pd.DataFrame(train_data["data"]).iloc[0]


#txt = (train_data["data"]).iloc[0]

txt = sample_context


In [ ]:
print("--- original ---")
print(txt)
print("--- cleaning ---")
txt = re.sub(r'[^\w\s]', '', str(txt).lower().strip())
print(txt)
print("--- tokenization ---")
txt = txt.split()
print(txt)

--- original ---
Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
--- cleaning ---
beyoncé giselle knowlescarter biːˈjɒnseɪ beeyonsay born september 4 1981 is an american singer songwriter record producer and actress born and raised in houston texas she performed in various singing and dancing competitions as a child and rose to fame in the late 1990s 

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
lst_stopwords = nltk.corpus.stopwords.words("english")
lst_stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
print("--- remove stopwords ---")
txt = [word for word in txt if word not in lst_stopwords]
print(txt)

--- remove stopwords ---
['beyoncé', 'giselle', 'knowlescarter', 'biːˈjɒnseɪ', 'beeyonsay', 'born', 'september', '4', '1981', 'american', 'singer', 'songwriter', 'record', 'producer', 'actress', 'born', 'raised', 'houston', 'texas', 'performed', 'various', 'singing', 'dancing', 'competitions', 'child', 'rose', 'fame', 'late', '1990s', 'lead', 'singer', 'rb', 'girlgroup', 'destinys', 'child', 'managed', 'father', 'mathew', 'knowles', 'group', 'became', 'one', 'worlds', 'bestselling', 'girl', 'groups', 'time', 'hiatus', 'saw', 'release', 'beyoncés', 'debut', 'album', 'dangerously', 'love', '2003', 'established', 'solo', 'artist', 'worldwide', 'earned', 'five', 'grammy', 'awards', 'featured', 'billboard', 'hot', '100', 'numberone', 'singles', 'crazy', 'love', 'baby', 'boy']


In [ ]:
#str1 = ""
#txt = str1.join(txt)
#txt

In [ ]:

from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer()

In [ ]:
#print(lemmatizer.lemmatize(txt))

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()


In [ ]:
for word in sample_context.split():
  print(ps.stem(word))

beyoncé
gisel
knowles-cart
(/biːˈjɒnseɪ/
bee-yon-say)
(born
septemb
4,
1981)
is
an
american
singer,
songwriter,
record
produc
and
actress.
born
and
rais
in
houston,
texas,
she
perform
in
variou
sing
and
danc
competit
as
a
child,
and
rose
to
fame
in
the
late
1990
as
lead
singer
of
r&b
girl-group
destiny'
child.
manag
by
her
father,
mathew
knowles,
the
group
becam
one
of
the
world'
best-sel
girl
group
of
all
time.
their
hiatu
saw
the
releas
of
beyoncé'
debut
album,
danger
in
love
(2003),
which
establish
her
as
a
solo
artist
worldwide,
earn
five
grammi
award
and
featur
the
billboard
hot
100
number-on
singl
"crazi
in
love"
and
"babi
boy".


In [ ]:
train_data_copy = train_data.copy()

train_data_copy['word_count'] = train_data_copy["data"].apply(lambda x: len(str(x).split(" ")))
train_data_copy['char_count'] = train_data_copy["data"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))
train_data_copy['sentence_count'] = train_data_copy["data"].apply(lambda x: len(str(x).split(".")))
train_data_copy['avg_word_length'] = train_data_copy['char_count'] / train_data_copy['word_count']
train_data_copy['avg_sentence_lenght'] = train_data_copy['word_count'] / train_data_copy['sentence_count']
train_data_copy.head()

,version,data,word_count,char_count,sentence_count,avg_word_length,avg_sentence_lenght
0,v2.0,"{'title': 'Beyoncé', 'paragraphs': [{'qas': [{...",25163,177951,524,7.071931,48.020992
1,v2.0,"{'title': 'Frédéric_Chopin', 'paragraphs': [{'...",24264,171911,583,7.085023,41.619211
2,v2.0,{'title': 'Sino-Tibetan_relations_during_the_M...,15916,102075,351,6.413358,45.344729
3,v2.0,"{'title': 'IPod', 'paragraphs': [{'qas': [{'qu...",12663,86994,312,6.869936,40.586538
4,v2.0,{'title': 'The_Legend_of_Zelda:_Twilight_Princ...,12257,89601,198,7.310190,61.904040
